# Model-based control using Extended Dynamic Mode Decomposition


In this tutorial we will be exploring the Extended Dynamic Mode Decomposition (EDMD) with control on the Van der Pol oscillator. The goal is to provide an understanding of how EDMD can be used to model and control dynamic systems. The Van der Pol oscillator, a classic example of non-linear oscillation, serves as a simple system. The control feedback will be implemented using the classical Linear Quadratic Regulator (LQR) method. LQR finds the optimal control input that drives the system from an initial state to a desired target state while minimizing the cost function. 

You can learn how EDMD and LQR can be used together to model and control a non-linear system. In this way EDMD can be an effective alternative to traditional methods for controlling dynamic systems.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from tqdm import tqdm

from datafold import EDMD, InitialCondition, TSCDataFrame, TSCTransformerMixin
from datafold.appfold.mpc import LQR
from datafold.dynfold.dmd import DMDControl
from datafold.utils._systems import VanDerPol

In [ ]:
# Note that it may not converge when choosing another seed (defaults to 55)
rng = np.random.default_rng(55)

Set up the Van der Pol system and specify parameters for the sampling procedure. In the system we only control the $y$-direction.

In [ ]:
vdp = VanDerPol(control_coord="y")

n_timeseries = 20  # number of timeseries in training set
n_timesteps = 200  # how many timesteps for every time series
dt = 0.01  # delta time
time_values = np.arange(0, n_timesteps * dt, dt)

Set up the initial conditions, which are uniformly sampled $x_{ic}\in[-3,3]^2$ and the control input, which is uniformly sampled $u_{ic}\in[-3,3]$ and kept constant for the over the prediction horizon.

The perform the prediction with the original Van der Pol model.

In [ ]:
X_ic = rng.uniform(-3.0, 3.0, size=(n_timeseries, 2))
idx = pd.MultiIndex.from_arrays([np.arange(n_timeseries), np.zeros(n_timeseries)])
X_ic = TSCDataFrame(X_ic, index=idx, columns=vdp.feature_names_in_)

U_tsc = rng.uniform(-3.0, 3.0, size=(n_timeseries, 1, 1))
U_tsc = np.tile(U_tsc, (1, n_timesteps - 1, 1))
U_tsc = TSCDataFrame.from_tensor(
    U_tsc,
    time_series_ids=X_ic.ids,
    feature_names=vdp.control_names_in_,
    time_values=time_values[:-1],
)

In [ ]:
X_tsc, U_tsc = vdp.predict(X_ic, U=U_tsc)

View the time series collecion data (state and control) from the original model and also plot it afterwards. 
This data is the basis to construct the Extended Dynamic Mode Decomposition model below.

In [ ]:
X_tsc

In [ ]:
U_tsc

In [ ]:
plt.figure(figsize=(10, 7))

for i in X_tsc.ids:
    idx = pd.IndexSlice[i, :]
    plt.plot(X_tsc.loc[idx, "x1"].to_numpy(), X_tsc.loc[idx, "x2"].to_numpy())
plt.title("Time series collection (training data)");

### Set up and fit the EDMD model to training data

For this we first specify the dictionary, which here is an own datafold transformer class. Afterwards we specify the EDMD model with the dictionary as well es a DMD method that handles control. Finally, to constrct the model, we fit the data to the sampled training data.

In [ ]:
class VdPDictionary(BaseEstimator, TSCTransformerMixin):
    def _more_tags(self):
        return dict(tsc_contains_orig_states=True)

    def get_feature_names_out(self, input_features=None):
        return ["x1^2", "x1^2 * x2"]

    def fit(self, X, y=None):
        self._setup_feature_attrs_fit(X)
        return self

    def transform(self, X: TSCDataFrame):
        X = X.copy()
        X["x1^2"] = np.square(X.loc[:, "x1"].to_numpy())
        X["x1^2 * x2"] = X["x1^2"].to_numpy() * X["x2"].to_numpy()
        return X

In [ ]:
edmd = EDMD(
    dict_steps=[("vdpdict", VdPDictionary())],
    dmd_model=DMDControl(),
    include_id_state=False,
)
edmd.fit(X_tsc, U=U_tsc)

We can now continue to perform out-of-sample predictions in which we use the EDMD model and generate control input from a linear quadratic regulator. 

In [ ]:
# number of time steps and time values for controlled time series
n_timesteps_oos = 500
time_values_oos = np.linspace(0, n_timesteps_oos * X_tsc.delta_time, n_timesteps_oos)

# random initial state
X_ic_oos = rng.uniform(-3, 3, size=(1, 2))
X_ic_oos = InitialCondition.from_array(
    X_ic_oos, feature_names=edmd.feature_names_in_, time_value=0
)

target_state = InitialCondition.from_array(
    np.array([0, 0]), feature_names=edmd.feature_names_in_, time_value=0
)

The target state is the origin of the Van der Pol phase space. We also provide a cost of input as well as the discrepancy of state. 

In [ ]:
lqr = LQR(edmd=edmd, cost_running=np.array([1, 1, 0, 0]), cost_input=1e-2)
lqr.preset_target_state(target_state);

Now we can perform a closed-loop simulation, where the control input is adjusted based on the system's output in the current time step. Therefore, the model's state is continuously monitored and used to update the control input, allowing the system to adjust its behavior based on the current state of the system.

In [ ]:
# allocate data structures and fill in the following system loop
X_oos = TSCDataFrame.from_array(
    np.zeros((n_timesteps_oos, 2)),
    feature_names=vdp.feature_names_in_,
    time_values=time_values_oos,
)
U_oos = TSCDataFrame.from_array(
    np.zeros((n_timesteps_oos - 1, 1)),
    feature_names=vdp.control_names_in_,
    time_values=time_values_oos[:-1],
)

X_oos.iloc[0, :] = X_ic_oos.to_numpy()

for i in tqdm(range(1, n_timesteps_oos)):
    state = X_oos.iloc[[i - 1], :]
    U_oos.iloc[i - 1, :] = lqr.control_sequence(X=state)
    new_state, _ = vdp.predict(
        state, U=U_oos.iloc[[i - 1], :], time_values=time_values_oos[i - 1 : i + 1]
    )
    X_oos.iloc[i, :] = new_state.iloc[[1], :].to_numpy()

For comparison we simulate the system without any control input. 

In [ ]:
trajectory_uncontrolled, _ = vdp.predict(
    X_ic_oos, U=np.zeros((n_timesteps_oos - 1)), time_values=time_values_oos
)

Finally, we plot the controlled trajectory and compare it to the uncontrolled system. The blue arrows highlight the control input along the vertical direction at the respective state. We see that that the system is able to reach the origin as the specified target state. 

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(
    X_oos.loc[:, "x1"].to_numpy(),
    X_oos.loc[:, "x2"].to_numpy(),
    c="red",
    label="controlled traj.",
)
plt.quiver(
    *X_oos.to_numpy()[:-1, :].T,
    *np.column_stack(
        [np.zeros_like(U_oos.to_numpy()), U_oos.to_numpy() / X_oos.delta_time]
    ).T,
    color="blue",
    label="control"
)
plt.plot(X_oos.iloc[0, 0], X_oos.iloc[0, 1], "o", c="red")
plt.plot(
    trajectory_uncontrolled.loc[:, "x1"].to_numpy(),
    trajectory_uncontrolled.loc[:, "x2"].to_numpy(),
    c="black",
    label="uncontrolled traj.",
)
plt.plot(
    trajectory_uncontrolled.iloc[0, 0],
    trajectory_uncontrolled.iloc[0, 1],
    "o",
    c="black",
    label="initial state",
)
plt.plot(
    target_state.iloc[0, 0],
    target_state.iloc[0, 1],
    "*",
    c="black",
    label="target state",
)
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.grid()

Here we plot the trajectory and its norm over time on its path to the target state.

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(X_oos.time_values(), X_oos.loc[:, "x1"].to_numpy(), c="black", label="x1")
plt.plot(
    X_oos.time_values(),
    X_oos.loc[:, "x2"].to_numpy(),
    c="blue",
    label="x2 (controlled)",
)
plt.xlabel("t")
plt.legend()
plt.ylabel("x1, x2")
plt.grid()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.linalg.norm(X_oos.to_numpy(), axis=1), c="blue", label="state")
plt.axhline(np.linalg.norm(target_state), c="red", label="target")
plt.xlabel("t")
plt.ylabel("norm")
plt.grid()
plt.legend();